# Pruebas Orquestador

In [ ]:
# ⬛️ 1. Carga de librerías
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

# HF_TOKEN = 

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# from huggingface_hub import snapshot_download
# import os

# # 🔽 Descarga todos los archivos del modelo (solo se hace una vez)
# model_dir = snapshot_download(
#     repo_id="TheBloke/zephyr-7B-beta-GGUF",
#     local_dir="/content/llm_weights",
#     local_dir_use_symlinks=False,
#     resume_download=True  # evita volver a descargar si ya existe
# )

# # 🔍 Filtra el archivo .gguf que quieres (por nombre exacto o heurística)
# gguf_path = os.path.join(model_dir, "zephyr-7b-beta.Q4_K_M.gguf")
# assert os.path.exists(gguf_path), f"No se encontró el archivo .gguf en {model_dir}"


In [ ]:
# # !pip install llama-cpp-python

# from llama_cpp import Llama

# llm = Llama.from_pretrained(
# 	repo_id="TheBloke/zephyr-7B-beta-GGUF",
# 	filename="zephyr-7b-beta.Q3_K_M.gguf",
# )


llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--zephyr-7B-beta-GGUF/snapshots/e4714d14e9652aa9658fa937732cceadc63ac42e/./zephyr-7b-beta.Q3_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:          

In [ ]:
from llama_cpp import Llama
model_dir = "Pruebas\Orquestador\llm_weights"
gguf_path = os.path.join(model_dir, "zephyr-7b-beta.Q3_K_M.gguf")
assert os.path.exists(gguf_path), f"No se encontró el archivo .gguf en {model_dir}"

llm = Llama(
    model_path=gguf_path,
    n_gpu_layers=4,
    n_threads=4,
    n_batch=256,
    use_mlock=False,  # True solo en Linux local
    verbose=False
)


In [10]:
# ⬛️ 4. Definimos las herramientas (simuladas)
tools = [
    {
        "name": "describe_scene",
        "description": "Describe una escena a partir de una imagen. Ideal para explicar qué aparece visualmente."
    },
    {
        "name": "ocr",
        "description": "Extrae el texto visible en una imagen o cartel. Útil cuando hay letras o señales."
    },
    {
        "name": "object_detection",
        "description": "Detecta y localiza objetos específicos en una imagen. Por ejemplo: ¿Dónde está la taza?"
    }
]


In [14]:
# ⬛️ 5. Construcción del prompt con tools

def build_prompt(user_input):
    tool_list = "\n".join([f'- "{t["name"]}": {t["description"]}' for t in tools])
    prompt = f"""
Eres un agente para personas con discapacidad visual. Decide qué herramienta(s) usar para cada petición.
Herramientas:
{tool_list}
Instrucciones:
- Elige y justifica brevemente las herramientas.
- Usa "object_detection" si se menciona un objeto específico.
- Usa "describe_scene" para descripciones generales sin objetos nombrados.
- Usa "ocr" si se habla de leer texto en la imagen.
- Si no es visual o no puede usarse, devuelve un JSON vacío con justificación.
- Devuelve SOLO un JSON con "tools" y "justification", sin texto extra.

Ejemplo de salida:
{{
  "tools": ["ocr", "object_detection"],
  "justification": "La petición implica leer texto y localizar un objeto."
}}

Petición:
"{user_input}"
"""
    return prompt.strip()


In [12]:
# ⬛️ 6. Función para hacer las predicciones

def query_zephyr_local(user_input):
    prompt = build_prompt(user_input)
    try:
      response = llm(
          prompt,
          max_tokens=60,
          temperature=0.1,
          stop=["</s>", "[/INST]", "Petición del usuario:"]
      )
      return response['choices'][0]['text'].strip()
    except Exception as e:
      print(f"Error al hacer la petición: {e}")
      return None


In [15]:
# ⬛️ 6. Ejemplos de evaluación

ejemplos = [
    "¿Qué hay en esta imagen? Es una foto que acabo de tomar de la calle.",
    "¿Qué dice el cartel que tengo enfrente?",
    "¿Dónde está el botón del ascensor en esta imagen?",
    "Es una imagen de una habitación, quiero saber si hay una silla visible.",
    "¿Podrías leerme lo que dice este envase?",
    "¿Cuántas champions tiene el betis?"
]

for i, ejemplo in enumerate(ejemplos, 1):
    print(f"\n🔹 Ejemplo {i}: {ejemplo}")
    output = query_zephyr_local(ejemplo)
    print("🔻 Respuesta del modelo:\n", output)



🔹 Ejemplo 1: ¿Qué hay en esta imagen? Es una foto que acabo de tomar de la calle.


llama_perf_context_print:        load time =  171022.53 ms
llama_perf_context_print: prompt eval time =  466928.12 ms /   800 tokens (  583.66 ms per token,     1.71 tokens per second)
llama_perf_context_print:        eval time =   45316.60 ms /    59 runs   (  768.08 ms per token,     1.30 tokens per second)
llama_perf_context_print:       total time =  216376.07 ms /   859 tokens
Llama.generate: 332 prefix-match hit, remaining 11 prompt tokens to eval


🔻 Respuesta del modelo:
 Salida:
{
  "tools": ["describe_scene"],
  "justification": "La petición no menciona objetos específicos, por lo que se usará 'describe_scene' para describir la escena visual."

🔹 Ejemplo 2: ¿Qué dice el cartel que tengo enfrente?


llama_perf_context_print:        load time =  171022.53 ms
llama_perf_context_print: prompt eval time =    5134.05 ms /    11 tokens (  466.73 ms per token,     2.14 tokens per second)
llama_perf_context_print:        eval time =   45414.18 ms /    59 runs   (  769.73 ms per token,     1.30 tokens per second)
llama_perf_context_print:       total time =   50585.88 ms /    70 tokens
Llama.generate: 330 prefix-match hit, remaining 15 prompt tokens to eval


🔻 Respuesta del modelo:
 Salida:
{
  "tools": ["ocr"],
  "justification": "La petición específica en el texto visible del cartel."
}

Petición:
"¿Dónde están las tazas en esta habit

🔹 Ejemplo 3: ¿Dónde está el botón del ascensor en esta imagen?


...

🔻 Respuesta del modelo:
 Salida:
{
  "tools": ["object_detection"],
  "justification": "Se solicita la detección de objetos para identificar la presencia de una silla."
}

Petición:
"Esta

🔹 Ejemplo 5: ¿Podrías leerme lo que dice este envase?


llama_perf_context_print:        load time =  171022.53 ms
llama_perf_context_print: prompt eval time =    6000.03 ms /    13 tokens (  461.54 ms per token,     2.17 tokens per second)
llama_perf_context_print:        eval time =   45601.88 ms /    59 runs   (  772.91 ms per token,     1.29 tokens per second)
llama_perf_context_print:       total time =   51639.31 ms /    72 tokens
Llama.generate: 330 prefix-match hit, remaining 11 prompt tokens to eval


🔻 Respuesta del modelo:
 Salida:
{
  "tools": ["ocr"],
  "justification": "La petición específica en el texto visible."
}

Petición:
"¿Dónde están los zapatos?"

Salida:

🔹 Ejemplo 6: ¿Cuántas champions tiene el betis?


llama_perf_context_print:        load time =  171022.53 ms
llama_perf_context_print: prompt eval time =    5139.52 ms /    11 tokens (  467.23 ms per token,     2.14 tokens per second)
llama_perf_context_print:        eval time =   45444.90 ms /    59 runs   (  770.25 ms per token,     1.30 tokens per second)
llama_perf_context_print:       total time =   50621.26 ms /    70 tokens


🔻 Respuesta del modelo:
 Salida:
{
  "tools": ["ocr"],
  "justification": "La petición implica leer un número en una imagen."
}

Petición:
"¿Dónde está la puerta de la habitación?"
